In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import torch
import torch.onnx
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
from tqdm import tqdm
import time

In [2]:
df = pd.read_csv("ISIC_2019_Training_GroundTruth.csv")
df

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
25326,ISIC_0073247,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
25327,ISIC_0073248,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
25328,ISIC_0073249,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25329,ISIC_0073251,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
#df['image_path'] = df['image_path'].apply(lambda x: x.split('/')[-1])
#df['image_path'] = df['image_path'].apply(lambda x: x.split("\\")[-1])
#df['image_path']

In [3]:
image_folder = 'ISIC_2019_Training_Input/ISIC_2019_Training_Input'
df['image_path'] = df['image'].apply(lambda x: os.path.join(image_folder, f"{x}.jpg"))

# One-hot encoded labels
labels = df.columns[1:-1].tolist()

# Split the dataset
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df[labels].idxmax(axis=1))

In [4]:
train_df

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK,image_path
11880,ISIC_0033283,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ISIC_2019_Training_Input/ISIC_2019_Training_In...
14068,ISIC_0055317,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ISIC_2019_Training_Input/ISIC_2019_Training_In...
14712,ISIC_0056356,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ISIC_2019_Training_Input/ISIC_2019_Training_In...
23632,ISIC_0070561,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ISIC_2019_Training_Input/ISIC_2019_Training_In...
20851,ISIC_0066198,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ISIC_2019_Training_Input/ISIC_2019_Training_In...
...,...,...,...,...,...,...,...,...,...,...,...
9130,ISIC_0030533,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,ISIC_2019_Training_Input/ISIC_2019_Training_In...
20008,ISIC_0064832,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ISIC_2019_Training_Input/ISIC_2019_Training_In...
4858,ISIC_0026261,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ISIC_2019_Training_Input/ISIC_2019_Training_In...
15151,ISIC_0057057,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ISIC_2019_Training_Input/ISIC_2019_Training_In...


In [5]:
valid_df

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK,image_path
7574,ISIC_0028977,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,ISIC_2019_Training_Input/ISIC_2019_Training_In...
10335,ISIC_0031738,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,ISIC_2019_Training_Input/ISIC_2019_Training_In...
15337,ISIC_0057361,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,ISIC_2019_Training_Input/ISIC_2019_Training_In...
15731,ISIC_0058026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,ISIC_2019_Training_Input/ISIC_2019_Training_In...
3973,ISIC_0025376,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,ISIC_2019_Training_Input/ISIC_2019_Training_In...
...,...,...,...,...,...,...,...,...,...,...,...
4411,ISIC_0025814,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ISIC_2019_Training_Input/ISIC_2019_Training_In...
19982,ISIC_0064796,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,ISIC_2019_Training_Input/ISIC_2019_Training_In...
6912,ISIC_0028315,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ISIC_2019_Training_Input/ISIC_2019_Training_In...
10757,ISIC_0032160,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ISIC_2019_Training_Input/ISIC_2019_Training_In...


In [6]:
from PIL import Image

class ISICDataset(Dataset):
    def __init__(self, dataframe, labels, transform=None):
        self.dataframe = dataframe
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image = Image.open(row['image_path']).convert('RGB')
        label = torch.tensor(row[self.labels].values.astype(float), dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, label

In [7]:
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [8]:
train_dataset = ISICDataset(train_df, labels, transform=transform_train)
valid_dataset = ISICDataset(valid_df, labels, transform=transform_valid)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)


In [9]:
# Check if CUDA is available and set the device accordingly
device = torch.device("cuda")
print(f"Using device: {device}")

Using device: cuda


In [15]:
import torch
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))


PyTorch Version: 2.5.1+cu118
CUDA Available: True
GPU Name: NVIDIA GeForce RTX 4070 Laptop GPU


In [11]:
import torch.nn as nn
from torchvision import models

class ModifiedEfficientNetB3(nn.Module):
    def __init__(self, num_classes, dropout_prob=0.5):
        super(ModifiedEfficientNetB3, self).__init__()
        # Load pretrained EfficientNet-B3
        self.efficientnet = models.efficientnet_b3(weights=models.EfficientNet_B3_Weights.IMAGENET1K_V1)
        # Freeze the feature extractor layers if needed (optional)
        for param in self.efficientnet.parameters():
            param.requires_grad = True
        # Modify the classifier head
        self.efficientnet.classifier = nn.Sequential(
            nn.Dropout(p=dropout_prob),                     # Dropout layer
            nn.Linear(self.efficientnet.classifier[1].in_features, 512),  # Intermediate fully connected layer
            nn.BatchNorm1d(512),                            # Batch normalization
            nn.ReLU(),                                      # ReLU activation
            nn.Linear(512, num_classes)                    # Final output layer
        )

    def forward(self, x):
        return self.efficientnet(x)

# Instantiate the modified EfficientNet-B3
num_classes = len(labels)  # Number of classes in the dataset
dropout_prob = 0.6  # Probability for dropout
efficientnet_model = ModifiedEfficientNetB3(num_classes=num_classes, dropout_prob=dropout_prob)
efficientnet_model = efficientnet_model.to(device)


In [13]:
optimizer = torch.optim.Adam(efficientnet_model.parameters(), lr=1e-4, weight_decay=1e-4)
criterion = torch.nn.BCEWithLogitsLoss()  # For multilabel classification

In [14]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.1, patience=2, verbose=True
)

c:\Users\bhuva\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [16]:
def calculate_accuracy(outputs, labels):
    # Apply sigmoid to get probabilities and convert to binary (multilabel threshold = 0.5)
    predictions = torch.sigmoid(outputs) > 0.5
    correct = (predictions == labels.bool()).sum().item()
    total = labels.numel()
    return correct / total

In [17]:
def train_model(model, criterion, optimizer, scheduler, train_loader, valid_loader, epochs=20):
    global best_loss, counter
    best_loss = float('inf')
    patience = 3  # For early stopping
    counter = 0

    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0
        train_acc = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_acc += calculate_accuracy(outputs, labels)

        # Validation phase
        model.eval()
        valid_loss = 0
        valid_acc = 0
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                valid_loss += loss.item()
                valid_acc += calculate_accuracy(outputs, labels)

        # Calculate average loss and accuracy
        avg_train_loss = train_loss / len(train_loader)
        avg_train_acc = train_acc / len(train_loader)
        avg_valid_loss = valid_loss / len(valid_loader)
        avg_valid_acc = valid_acc / len(valid_loader)

        # Print metrics
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train Loss: {avg_train_loss:.4f}, Train Accuracy: {avg_train_acc:.4f}")
        print(f"Valid Loss: {avg_valid_loss:.4f}, Valid Accuracy: {avg_valid_acc:.4f}")

        # Check for early stopping
        if avg_valid_loss < best_loss:
            best_loss = avg_valid_loss
            counter = 0
            torch.save(model.state_dict(), "best_model.pth")  # Save the best model
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping triggered.")
                break

        # Step the scheduler with validation loss
        scheduler.step(avg_valid_loss)

In [18]:
import time
start_time = time.time()

In [19]:

train_model(efficientnet_model, criterion, optimizer,scheduler, train_loader, valid_loader, epochs=10)
end_time = time.time()
time_diff = start_time - end_time
time_diff

Epoch 1/10
Train Loss: 0.2252, Train Accuracy: 0.9231
Valid Loss: 0.1627, Valid Accuracy: 0.9416
Epoch 2/10
Train Loss: 0.1396, Train Accuracy: 0.9463
Valid Loss: 0.1277, Valid Accuracy: 0.9516
Epoch 3/10
Train Loss: 0.1166, Train Accuracy: 0.9548
Valid Loss: 0.1146, Valid Accuracy: 0.9561
Epoch 4/10
Train Loss: 0.0968, Train Accuracy: 0.9628
Valid Loss: 0.1046, Valid Accuracy: 0.9589
Epoch 5/10
Train Loss: 0.0821, Train Accuracy: 0.9686
Valid Loss: 0.1018, Valid Accuracy: 0.9628
Epoch 6/10
Train Loss: 0.0681, Train Accuracy: 0.9743
Valid Loss: 0.0985, Valid Accuracy: 0.9631
Epoch 7/10
Train Loss: 0.0578, Train Accuracy: 0.9786
Valid Loss: 0.1016, Valid Accuracy: 0.9653
Epoch 8/10
Train Loss: 0.0505, Train Accuracy: 0.9813
Valid Loss: 0.0974, Valid Accuracy: 0.9651
Epoch 9/10
Train Loss: 0.0423, Train Accuracy: 0.9847
Valid Loss: 0.0981, Valid Accuracy: 0.9667
Epoch 10/10
Train Loss: 0.0359, Train Accuracy: 0.9871
Valid Loss: 0.1038, Valid Accuracy: 0.9669


-3839.6042189598083